<a href="https://colab.research.google.com/github/Samuele575/SMario_RL-AI_tutorialNick/blob/main/Mario_AI_model_with_PY_TutorialNick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUPER MARIO AI training model

Setup Mario in reading command in Python

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
# Import the game
import gym_super_mario_bros
#import JoypadSpace
from nes_py.wrappers import JoypadSpace
#import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100000):
  if done:
    # Start the game
    env.restart()
  # Do random action
  # (so:
  #     + state  -> is the frame of the game AFTER the action
  #     + reward -> POINT of the "maximization_function", this func is MAX when MARIO is the further to the right
  #     + done   -> Boolean saing IF the game is ended or is still going
  #     + info   -> metadata
  #)
  state, reward, done, info = env.step(env.action_space.sample())
  # Show game on the screen
  env.render()
# Close the game
env.close()

## Pre Process the env


In [ ]:
# Install PyTorch
!pip install torch trochvision

In [ ]:
# Install Stable Baseline3 for RL stuff
!pip install stable-baselines3[extra]

In [ ]:
# import Frame Stacker Wrapper and GrayScale Wrapper
import gym.wrappers import GrayScaleObservation
# import Vectorization Wrappers
import stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# import Matplotlib - to show the implat of frame stacking
from matplotlib import pyplot as plt

In [ ]:
# 1. Create the base env
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Greyscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Env
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

## Train the RL Model

Reinforcement Learning is a practice based on 4 element:

- Agent: the "element" controlled by the IA
- Actions: the movement the AI can do
- Environment: the set where the Agent is moving using action and thus changing sequentialy
- Reword: a number associated with the Action expressing the "goodness" of the move

In [ ]:
# Import os for file path management
import os
# Import PPO for algos
from stable_baselines3 import PPO
#import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
            n_steps=512)

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
model.save('thisisatestmodel')

## Test it Out

In [ ]:
# Load model
model = PPO.load('./train/best_model_1000000')

In [ ]:
state = env.reset()

In [ ]:
# Start the game
state = env.reset()
# Loop through the game
while True:

    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()